In [ ]:
#Verificar versión de gpu Nvidia
!nvidia-smi

In [ ]:
import cudf, cuml, cuxfilter
from pandas.io.json import json_normalize 
import pandas as pd 

In [ ]:
import time
#Fase de extracción y lectura con Rapids

#extracción y lectura con Pandas
pd_Pandas = pd.read_csv('df_data.csv',
                        names='uno')

#extraccion y lectura con Rapids
start = time.time()

gdf = cudf.read_csv('df_data.csv',
                    names='uno')

end = time.time()
caculatedCUDF = end - start

print("Cálculo de extracción y lectura con cudf gpu-Rapdis= {}".format(caculatedCUDF))

In [ ]:
#######Transformación con cudf##########

In [ ]:
import json
import time

cudf_Pandas = pd_Pandas

#crear formato json para despues crear un dataframe nuevo
start = time.time()
lista = list(cudf_Pandas['u'])

a = []
for i in range(0,len(lista)):
    d = lista[i].split('.gz:')
    a.append(json.loads((d[1])))
    
#Creo DataFrame con Pandas
pdPandas = pd.DataFrame(a)

# Convierto todo el dataframe a str
nuevo_pdPandas = pdPandas.applymap(str)

#Se crear un dataframe con cudf(propia de Rapids)
#gdf = cudf.DataFrame.from_pandas(nuevo_pdPandas)

#conviero para pandas dataframe(propia de Rapids)
#gdfPD = gdf.to_pandas()

#solo conservo los datos de la columna username
#mayor a 0
snNull = nuevo_pdPandas[nuevo_pdPandas.username.map(len) > 0]

#elimino username dicc y opencampus
snDicc = snNull[snNull.username != 'dicc']
snCampus = snDicc[snDicc.username != 'opencampus']

#convieto para cudf dataframe
gdf_new = cudf.DataFrame.from_pandas(snCampus)

#Eliminar columnas que no tiene mucha relevancia
gdf_new.drop_column('accept_language') #columna sin relevancia
gdf_new.drop_column('name') #muchos datos faltantes
gdf_new.drop_column('agent') #columna sin relevancia
gdf_new.drop_column('page') #muchos datos faltantes
gdf_new.drop_column('session') #muchos datos faltantes
gdf_new.drop_column('event') #datos dañados
gdf_new.drop_column('event_type') #columna sin relevancia

#eliminar filas con los datos perdidos
gdf_new.dropna(axis=0, how='any')

#Dividir columna context estaba en json
newContext = gdf_new['context'].tolist()
b=[]
for i in range(0,len(newContext)):
    d = newContext[i].split(',')
    b.append(d[1])

gdfContext = cudf.DataFrame()
gdfContext['General'] = b
gdfContext = gdfContext['General'].str.split()
gdfContext.drop_column(0)
gdfContext.columns=['userid_orgid']

#Unir
newData = gdf_new.join(gdfContext, how='left')
newData.drop_column('context')

#eliminar filas con datos perdidos
newData.dropna(axis=0, how='any')

#soltar duplicados
newData.sort_values('username') 
newData.drop_duplicates(subset='username', keep=False, inplace=False, ignore_index=False)


#dividir columna time
vtime = newData['time']
#vtime.str.normalize_spaces()
new_time = vtime.str.split(pat = "-")

day = new_time[2]
daytime = day.str.split(pat = 'T')
daytime.drop_column(1)
daytime.columns = ['day']

timenew = new_time.join(daytime, how='left')
timenew.drop_column(2)
timenew.columns = ['year','month','day']

#Unir todo al dataframe time al general
newData = newData.join(timenew)
newData.drop_column('time')

#eliminar filas con datos perdidos
newData.dropna(axis=0, how='any')

#resetear index
newData = newData.reset_index()
newData.drop_column('index')

end = time.time()
calculo_Transform= end - start

print('Cálculo de transformación con cudf gpu-Rapdis:', calculo_Transform)

In [ ]:
#Exploración de todo el Dataset con RAPIDS
print('Nombre de columnas:',newData.columns)
print('Dimension DataFrame:',newData.shape)
print('Tipos de datos de cada columna', newData.dtypes)
print('Información DATA general con Rapids:')
newData.head(2)

In [ ]:
###########Carga con Rapids#############

In [ ]:
#Guardar la data entendible en un nuevo csv
import time
start = time.time()

da_data = 'data_mejora.csv'
newData.to_csv(da_data)

end = time.time()
calculated = end - start

print("Cálculo carga con cudf gpu-Rapdis = {}".format(calculated))

In [ ]:
nueva_data = cudf.read_csv('data_mejora.csv')
nueva_data.drop_column('Unnamed: 0')

print('Tipos de datos de cada columna', newData.dtypes)
print('Información Data')
nueva_data.head(2)

In [ ]:
####Proceso Machine Learning##########

In [ ]:
import time
from cuml.preprocessing.model_selection import train_test_split

#Asignar dataset general a nuevas variables
gdf_dataX = nueva_data
gdf_dataY = nueva_data

#Dividir datasets en train y test
#datatrain 80% y datatest 20%
X_train, X_test, y_train, y_test = train_test_split(gdf_dataX, gdf_dataY, train_size=0.8)

#información
print(f'Dataset Original: {gdf_dataX.shape[0], gdf_dataY.shape[0]} elementos')
print(f'Data X_train: {X_train.shape[0]} elements')
print(f'Data X_test: {X_test.shape[0]} elements')
print(f'Data y_train: {y_train.shape[0]} elements')
print(f'Data y_test: {y_test.shape[0]} elements')

In [ ]:
import time
from cuml.preprocessing.LabelEncoder import LabelEncoder

inicio_Pre = time.time()
#Fase2 Preprocesamiento con cuml
#Convertir a variables categóricas

le = LabelEncoder()

#seleccionar columnas potenciales para work train de xTrain
value_usernameX = le.fit_transform(X_train.username)
value_EventSourceX = le.fit_transform(X_train.event_source)
value_DayX = le.fit_transform(X_train.day)

#seleccionar columnas potenciales para work train de xTest
value_usernameXT = le.fit_transform(X_test.username)
value_EventSourceXT = le.fit_transform(X_test.event_source)
value_DayXT = le.fit_transform(X_test.day)

#seleccionar columnas potenciales para work train de yTrain
value_usernameY = le.fit_transform(y_train.username)
value_EventSourceY = le.fit_transform(y_train.event_source)
value_DayY = le.fit_transform(y_train.day)

#seleccionar dos columnas potenciales para work train de yTest
value_usernameYT = le.fit_transform(y_test.username)
value_EventSourceYT = le.fit_transform(y_test.event_source)
value_DayYT = le.fit_transform(y_test.day)

fin_Pre = time.time()
calculo_Pre_ML = fin_Pre - inicio_Pre

print("Cálculo del proceso Machine Learning con cuml gpu-Rapdis = {}".format(calculo_Pre_ML))

In [ ]:
import time
import cupy as cp
from cuml.linear_model import LinearRegression

inicio_ML = time.time()
#Fase3 Uso de los algoritmos ML de cuml
#Algoritmo LinearRegression

#Creación de un nuevo dataframe Train 
dataTrain = cudf.DataFrame()
dataTrain['username'] = cp.asarray(value_usernameX, dtype=cp.float32)
dataTrain['day'] = cp.asarray(value_DayX, dtype=cp.float32)

#Creación de un nuevo dataframe Test
new_value = cp.asarray(value_usernameY, dtype=cp.float32)
dataTest = cudf.Series(new_value)

#Usar algoritmo LinearRegresssion de cuml
lr = LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = "eig")
reg = lr.fit(dataTrain, dataTest)

fin_ML = time.time()
calculo_Algorit_ML = fin_ML - inicio_ML

print("Algoritmo Regresión Lineal")
print("Coeficientes:")
print(reg.coef_)
print('')
print("Intercepción:")
print(reg.intercept_)
print('')
print('Predicción del modelo:')
print(lr.predict(dataTrain))
print('')

print("Cálculo del proceso Machine Learning con cuml gpu-Rapdis = {}".format(calculo_Algorit_ML))

In [ ]:
#Algoritmo regresión logistica de cuml
import time
import cupy as cp
from cuml.linear_model import LogisticRegression

inicio_RL = time.time()


#Creación de un nuevo dataframe Train 
dataTrain = cudf.DataFrame()
dataTrain['username'] = cp.asarray(value_usernameX, dtype=cp.float32)
dataTrain['day'] = cp.asarray(value_DayX, dtype=cp.float32)

#Creación de un nuevo dataframe Test
new_value = cp.asarray(value_usernameY, dtype=cp.float32)
dataTest = cudf.Series(new_value)

reg_lo = LogisticRegression()
reg_lo.fit(dataTrain,dataTest)

fin_RL = time.time()
calculo_RL = fin_RL - inicio_RL

print("Coefficients:")
print(reg_lo.coef_.to_output('cupy'))
print("Intercept:")
print(reg_lo.intercept_.to_output('cupy'))
print('Predicción del modelo:')
print(reg_lo.predict(dataTrain))
print('')

print("Cálculo del algoritmo Regresión Logistica con cuml gpu-Rapdis = {}".format(calculo_RL))

In [ ]:
#########Visualización###############

In [ ]:
from cuxfilter import DataFrame, themes, layouts
from cuxfilter.assets.custom_tiles import get_provider, Vendors

inicioV = time.time()
cux_df = cuxfilter.DataFrame.from_dataframe(dataTrain)

#crear gráfico de barras estadísticas
bar_chart_1 = cuxfilter.charts.bar('day', 'username', title='Gráfico de Barras Estadístico', data_points=5, add_interaction=False)
d = cux_df.dashboard([bar_chart_1])

finV = time.time()
calculo_V = finV - inicioV
print("Cálculo del proceso Visualización gpu-cuxfilter = {}".format(calculo_V))
print('')

bar_chart_1.view()

In [ ]:
#crear gráfico de líneas estadisticas
line_chart_1 = cuxfilter.charts.line('day', 'username', title='Gráfico de Lineas Estadístico', data_points=5, add_interaction=False)
dl = cux_df.dashboard([line_chart_1])

line_chart_1.view()

In [ ]:
#Gráfico general
d = cux_df.dashboard([bar_chart_1, line_chart_1], layout=cuxfilter.layouts.triple_feature, theme=cuxfilter.themes.rapids, title="Gráficos General Estadísticos")
await d.preview()

In [ ]:
####Detalles de todo el ambiente datascience####

In [ ]:
#Dividir columna course_user_tags
#newcontext.columns = ['course_user_tags','user_id','org_id']
#newcontext.head(2)

#dfnewcontext = newcontext["course_user_tags"].str.split(':', expand=True)
#dfnewcontext.sort_values(0)
#dfnewcontext.drop_duplicates(1, keep="last", inplace=True)
#dfnewcontext.drop(0, inplace=True)
#dfnewcontext.columns = ['user_id']
#indexNames = dfnewcontext[dfnewcontext['user_id'] == '{}' ].index
#nuevodf = dfnewcontext.drop(indexNames, inplace=True)
#nuevodf.reset_index()

In [ ]:
#Algoritmo Random Forest de cuml
#import cupy as cp
#from cuml.datasets.classification import make_classification
#from cuml.ensemble import RandomForestClassifier as cuRFC

#inicioRF = time.time()

#dataTrainUSER = cp.asarray(value_usernameX, dtype=cp.float32)
#dataTestUSER = cp.asarray(value_usernameY, dtype=cp.int32)


#dataTrainUSER, dataTestUSER = make_classification(n_samples=2, n_features=2,
#                                                 n_informative=2, n_redundant=0,
#                                                 random_state=0)

#cuml_model_RDF = cuRFC(max_features=1.0,
#                   n_bins=2,
#                   n_estimators=2)

#cuml_model_RDF.fit(dataTrainUSER, dataTestUSER)
#cuml_predict = cuml_model_RDF.predict(dataTrainUSER)

#finRF = time.time()
#calculoRF = finRF - inicioRF

#print('Predict labels:', cuml_predict)
#print('')

#print("Cálculo fase de entrenamiento algoritmo Random Forest con cuml = {}".format(calculoRF))

#Resultado erroneo